# Tutorial: Weaves
In this tutorial we demonstrate the creation of weaves using TexGen and how to import them into PuMApy. TexGen python scrips can be simply obtained by either following TexGen tutorials https://github.com/louisepb/TexGenScriptingGuide or by recording the actions in the TexGen GUI (of which there are already
bundled executables for Windows at https://sourceforge.net/projects/texgen/files/). 

Let's start by importing PuMApy, Numpy and TexGen and creating a simple LayerToLayer weave using a TexGen python script. 

In [ ]:
# Run this line only the first time you open a tutorial in Google Colab
pip install 'git+https://github.com/nasa/puma.git'

In [1]:
import os
pumadir = os.path.abspath('..')
import pumapy as puma
import numpy as np
from TexGen.Core import *

In [ ]:
NumBinderLayers = 2 
NumXYarns = 3 
NumYYarns = 4 
XSpacing = 1.0
YSpacing = 1.0 
XHeight = 0.2 
YHeight = 0.2 
weave = CTextileLayerToLayer(NumXYarns, NumYYarns, XSpacing, YSpacing, XHeight, YHeight, NumBinderLayers)

#set number of binder / warp yarns
NumBinderYarns = 2
NumWarpYarns = NumXYarns - NumBinderYarns
weave.SetWarpRatio( NumWarpYarns )
weave.SetBinderRatio( NumBinderYarns )

#setup layers: 3 warp, 4 weft
weave.SetupLayers( 3, 4, NumBinderLayers)

#set yarn dimensions: widths / heights
weave.SetYYarnWidths( 0.8 )
weave.SetYYarnWidths( 0.8 )
weave.SetBinderYarnWidths( 0.4 )
weave.SetBinderYarnHeights( 0.1 )

#define offsets for the two binder yarns
P = [[0, 1, 3, 0],[3, 0, 0, 3]]

#assign the z-positions to the binder yarns
for y in range(NumWarpYarns,NumXYarns): #loop through number of binder yarns
	offset = 0 
	for x in range(0,NumYYarns): #loop through the node positions	
		weave.SetBinderPosition(x, y, P[y-NumWarpYarns][offset])
		offset = offset+1

Next we create the domain, i.e. the lengths of the box containing the weave. In this case we will let TexGen figure it out automatically using the AssignDefaultDomain function:

In [ ]:
weave.AssignDefaultDomain()
domain = weave.GetDefaultDomain()

Then, we pass the weave and the domain to the PuMApy exporter, which creates the voxel grid and exports it to a directory (expensive step ~1.5-2 mins):

In [ ]:
puma.export_weave_vtu(pumadir + "/tests/out/weavetest", weave, domain, 200)

This can be then read back into a pumapy.Workspace:

In [ ]:
ws = puma.import_weave_vtu(pumadir + "/tests/out/weavetest")

And we can volume render it (remember to close the popup window with 'q', not directly):

In [ ]:
puma.render_volume(ws, cutoff=(1, np.unique(ws.matrix).size))